In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

## CapsNet layers

def squash(vectors, axis=-1):
    s_squared_norm = tf.reduce_sum(tf.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / tf.sqrt(s_squared_norm + 1e-7)
    return scale * vectors

class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return tf.sqrt(tf.reduce_sum(tf.square(inputs), -1) + 1e-7)

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]
        self.W = self.add_weight(shape=[self.num_capsule, self.input_num_capsule,
                                        self.dim_capsule, self.input_dim_capsule],
                                 initializer='glorot_uniform',
                                 name='W')

    def call(self, inputs, training=None):
        u_hat = tf.einsum('...ji,jik->...jk', inputs, self.W)
        
        b = tf.zeros(shape=[tf.shape(inputs)[0], self.input_num_capsule, self.num_capsule])
        
        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=1)
            outputs = squash(tf.einsum('...ij,...jk->...ik', c, u_hat))
            if i < self.routings - 1:
                b += tf.einsum('...ik,...jk->...ij', outputs, u_hat)
        
        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

## CapsNet model

def CapsNet(input_shape, n_class, routings):
    x = layers.Input(shape=input_shape)
    
    # Layer 1: Conv2D
    conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu')(x)
    
    # Layer 2: Conv2D
    conv2 = layers.Conv2D(filters=256, kernel_size=9, strides=2, padding='valid', activation='relu')(conv1)
    
    # Layer 3: Primary Caps
    primarycaps = layers.Conv2D(filters=32, kernel_size=9, strides=2, padding='valid', activation='relu')(conv2)
    primarycaps_reshaped = layers.Reshape((-1, 8))(primarycaps)
    primarycaps_squashed = layers.Lambda(squash)(primarycaps_reshaped)
    
    # Layer 4: Digit Caps
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings)(primarycaps_squashed)
    
    # Layer 5: Output
    out_caps = Length()(digitcaps)
    
    # Models for training and evaluation (prediction)
    model = models.Model(x, out_caps)
    
    return model

## Data Preprocessing

def load_and_preprocess_data(data_dir, img_size=(64, 64)):
    classes = ['cloudy', 'desert', 'green_area', 'water']
    X = []
    y = []
    
    for class_idx, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = Image.open(img_path).convert('RGB')
            img = img.resize(img_size)
            img_array = np.array(img) / 255.0  # Normalize to [0, 1]
            X.append(img_array)
            y.append(class_idx)
    
    return np.array(X), np.array(y)

# Load and preprocess the data
data_dir = '/kaggle/input/remotesensing/Remote sensing satellite images dataset for objects detection/rssid'  # Update this to your Kaggle dataset path
X, y = load_and_preprocess_data(data_dir)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoded format
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Model Configuration
input_shape = (64, 64, 3)
n_class = 4  # cloudy, desert, green_area, water
routings = 3

# Create and compile the model
model = CapsNet(input_shape, n_class, routings)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

## Training
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

## Evaluation
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

## Visualization
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

## Predictions
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Classification Report
class_names = ['cloudy', 'desert', 'green_area', 'water']
print(classification_report(true_classes, predicted_classes, target_names=class_names))

# Sample predictions
n_samples = 5
sample_indices = np.random.choice(len(X_test), n_samples, replace=False)

plt.figure(figsize=(15, 3))
for i, idx in enumerate(sample_indices):
    plt.subplot(1, n_samples, i+1)
    plt.imshow(X_test[idx])
    true_label = class_names[true_classes[idx]]
    pred_label = class_names[predicted_classes[idx]]
    plt.title(f"True: {true_label}\nPred: {pred_label}")
    plt.axis('off')
plt.tight_layout()
plt.show()

Model: "functional_39"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_19 (InputLayer)     │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_58 (Conv2D)              │ (None, 56, 56, 256)    │        62,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 24, 24, 256)    │     5,308,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_60 (Conv2D)              │ (None, 8, 8, 32)       │       663,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_19 (Reshape)            │ (None, 256, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_19 (Lambda)              │ (None, 256, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capsule_layer_19 (CapsuleLayer) │ (None, 4, 16)          │       131,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ length_19 (Length)              │ (None, 4)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,165,792 (23.52 MB)

 Trainable params: 6,165,792 (23.52 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


ValueError: Exception encountered when calling CapsuleLayer.call().

[1mShape must be rank 3 but is rank 4
	 for 1th input and equation: ...ji,jik->...jk for '{{node functional_39_1/capsule_layer_19_1/einsum/Einsum}} = Einsum[N=2, T=DT_FLOAT, equation="...ji,jik->...jk"](functional_39_1/lambda_19_1/mul, functional_39_1/capsule_layer_19_1/einsum/Einsum/ReadVariableOp)' with input shapes: [?,256,8], [4,256,16,8].[0m

Arguments received by CapsuleLayer.call():
  • inputs=tf.Tensor(shape=(None, 256, 8), dtype=float32)
  • training=True